In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re

from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback,BartTokenizerFast

)
from datasets import Dataset
from tqdm import tqdm

In [3]:
model_checkpoints = "checkpoint/encoder_decoder(4)freeze_MLM_test/checkpoint-192500"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [4]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (..) 제거
    #sentence = re.sub(r'[#@]+[가-힣A-Za-z#]+', ' ', sentence)
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    # 스폐셜 토큰 적용할 거면 여기 위에 영어 외 문자 공백으로 만들때 스폐셜 토큰을 넘어갈 수 있도록 지정해주면된다.
    # 그리고 세번째 정규표현식을 지워야 할 것이다. 
    return sentence

In [5]:
train_df = pd.read_csv('data/train_total.csv')
val_df = pd.read_csv('data/val_total.csv')

In [6]:
train_df.head()

,Id,Text,Summary,Category
0,fd321028-d5b4-55f7-9e20-2eaa262f9154,"['그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까?', '우리 비행...","비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.",상거래(쇼핑)
1,c51be2e4-c8d0-5cea-b1ae-cde1fe8f8ab6,"['Kf마스크만 5부제 하는거지?', '응. 면마스크는 아무때나 사도될껀?', '면...",비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.,상거래(쇼핑)
2,e90e721f-00d1-5114-aa5d-5f1061472a29,['아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애'...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...,상거래(쇼핑)
3,b215f3a2-d647-59f9-8410-1274ee5edd97,"['칫솔사야하는데 쓱으로 살까?', '뭘 칫솔사는것까지 물어보시남ㅋㅋㅋ', '아 그...",칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.,상거래(쇼핑)
4,0bda61b6-1396-5a2a-a049-0b4035e40d59,"['잠도안오네ㅐ얼릉 고구마츄 먹고싶단', '그게 그렇게 맛있었어??? 아주 여보 빼...",잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.,상거래(쇼핑)


In [7]:
# 전체 Text 데이터에 대한 전처리 (1)
train_text = []
train_summary = []

for tt in tqdm(train_df['Text']):
    train_text.append(preprocess_sentence(tt))

for ts in tqdm(train_df['Summary']):
      train_summary.append(preprocess_sentence(ts))

100%|███████████████████████████████████████████████████████████████| 279992/279992 [00:03<00:00, 86455.66it/s]


In [8]:
# 전체 Text 데이터에 대한 전처리 (1)
val_text = []
val_summary = []

for vt in tqdm(val_df['Text']):
    val_text.append(preprocess_sentence(vt))

for vs in tqdm(val_df['Summary']):
      val_summary.append(preprocess_sentence(vs))

100%|█████████████████████████████████████████████████████████████████| 35004/35004 [00:00<00:00, 81076.70it/s]


In [9]:
# 전체 Text 데이터에 대한 전처리 (1)
train_Category = []
val_Category= []

for tc in tqdm(train_df['Category']):
    train_Category.append(tc)

for vc in tqdm(val_df['Category']):
    val_Category.append(vc)


100%|███████████████████████████████████████████████████████████████| 35004/35004 [00:00<00:00, 2450223.92it/s]


In [10]:
for i in range(len(val_text)):
    temp_cat = "#"+val_Category[i]+"# "
    val_text[i] = temp_cat+val_text[i]
    
for i in range(len(train_text)):
    temp_cat = "#"+train_Category[i]+"# "
    train_text[i] = temp_cat+train_text[i]

In [11]:
train_df = pd.DataFrame(zip(train_text,train_summary), columns=['Text', 'Summary'])
val_df = pd.DataFrame(zip(val_text,val_summary), columns=['Text', 'Summary'])

In [12]:
train_df.head()

,Text,Summary
0,#상거래(쇼핑)# 그럼 날짜는 가격 큰 변동 없으면 6 28 7 13로 확정할까 우...,비행기 표 가격에 대해 이야기하며 특가 이벤트를 기다리고 있다
1,#상거래(쇼핑)# kf마스크만 5부제 하는거지 응 면마스크는 아무때나 사도될껀 면마...,비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다
2,#상거래(쇼핑)# 아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 ...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...
3,#상거래(쇼핑)# 칫솔사야하는데 쓱으로 살까 뭘 칫솔사는것까지 물어보시남 아 그 왕...,칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계 가자고 했다
4,#상거래(쇼핑)# 잠도안오네 얼릉 고구마츄 먹고싶단 그게 그렇게 맛있었어 아주 여보...,잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다


In [13]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)
test_samples = Dataset.from_pandas(val_df)

In [14]:
print(train_data)
print(val_data)
print(test_samples)

Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 279992)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 35004)
Dataset(features: {'Text': Value(dtype='string', id=None), 'Summary': Value(dtype='string', id=None)}, num_rows: 35004)


In [15]:
max_input = 128
max_target = 32
batch_size = 4
ignore_index = -100# tokenizer.pad_token_id

In [16]:
 def add_ignored_data(inputs, max_len, ignore_index):
        if len(inputs) < max_len:
            pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs

In [17]:
def add_padding_data(inputs, max_len):
        pad_index = tokenizer.pad_token_id
        if len(inputs) < max_len:
            pad = [pad_index] *(max_len - len(inputs))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:max_len]

        return inputs 

In [18]:
def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []

    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i], add_special_tokens=False), max_input))
    for i in range(len(data_to_process['Summary'])):
        label_id.append(tokenizer.encode(data_to_process['Summary'][i]))  
        label_id[i].append(tokenizer.eos_token_id)  
    for i in range(len(data_to_process['Summary'])):  
        dec_input_id = [tokenizer.eos_token_id]
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Summary'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

In [19]:
# special_words = [
#                 "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
#                 "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
#                 "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
#                 "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
#                 "#개인 및 관계#", "#미용과 건강#", "#상거래(쇼핑)#", "#시사/교육#", "#식음료#", 
#                 "#여가 생활#", "#일과 직업#", "#주거와 생활#", "#행사#",
#                 ]


# tokenizer.add_special_tokens({"additional_special_tokens": special_words})
model.resize_token_embeddings(len(tokenizer))

Embedding(30031, 768, padding_idx=3)

In [20]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

  0%|          | 0/280 [00:00<?, ?ba/s]

  0%|          | 0/36 [00:00<?, ?ba/s]

In [21]:
# # set special tokens
#from transformers import EncoderDecoderConfig
model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
model.config.max_length = 32 # 256은 쿠다 메모리 오류 생김
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 5
#model.config.suppress_tokens = [23782, 14338, 22554, 234]


In [22]:
model.config

BartConfig {
  "_name_or_path": "checkpoint/encoder_decoder(4)freeze_MLM_test/checkpoint-192500",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIV

In [23]:
rouge = Rouge()
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return rouge.get_scores(pred_str, label_str, avg=True)

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="checkpoint/freeze_pretrained_tf_test",
    num_train_epochs=10,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=64,
    learning_rate=3e-05,
    weight_decay=0.1,
    #label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    save_total_limit=3,
    load_best_model_at_end = True,
    logging_strategy = 'epoch',
    evaluation_strategy  = 'epoch',
    save_strategy ='epoch'

    # evaluation_strategy = "steps",# step별로 2버 loss가 오르는거 아니면 계속 반복하는듯
    #load_best_model_at_end = True,

)

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?
"""
DataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . 
또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.
tokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.
"""

'\nDataCollatorForSeq2Seq 를 사용하여 예제 배치를 생성 하십시오 . \n또한 일괄 처리에서 가장 긴 요소의 길이로 텍스트와 레이블을 동적으로 채워서 균일한 길이가 되도록 합니다.\ntokenizer를 설정하여 함수 에서 텍스트를 채울 수 있지만 padding=True동적 패딩이 더 효율적입니다.\n'

In [26]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
trainer.train()

/home/jx7789/anaconda3/envs/dohwan/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 279992
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 175000
  Number of trainable parameters = 123883776
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
1,2.096800,1.856583,"{'r': 0.2593372192400973, 'p': 0.26831419532059175, 'f': 0.2552187852189584}","{'r': 0.10661678669249798, 'p': 0.11225999682713486, 'f': 0.10522335067285649}","{'r': 0.24674493466549782, 'p': 0.2550383045204179, 'f': 0.24270169984873513}"
2,1.725700,1.800312,"{'r': 0.2764517315021707, 'p': 0.26539558290432447, 'f': 0.262106918846797}","{'r': 0.11581564152854451, 'p': 0.11264802418406492, 'f': 0.10988731532751417}","{'r': 0.2615366340557443, 'p': 0.25090868656911214, 'f': 0.24786460058858145}"
3,1.545800,1.784661,"{'r': 0.27932836236914305, 'p': 0.2696070241183919, 'f': 0.26546033966525173}","{'r': 0.11674878219703241, 'p': 0.11425068607060665, 'f': 0.11103629017107475}","{'r': 0.264616009780644, 'p': 0.255111784242901, 'f': 0.2513003640539408}"
4,1.408100,1.786517,"{'r': 0.2735497703781741, 'p': 0.27660762146294376, 'f': 0.2661154080825025}","{'r': 0.11373280952619491, 'p': 0.11696176389975389, 'f': 0.1109146319366768}","{'r': 0.25917738027866366, 'p': 0.2617371330784003, 'f': 0.25196090445841846}"
5,1.293300,1.810930,"{'r': 0.28038254046325617, 'p': 0.27174737998898063, 'f': 0.2671448728825775}","{'r': 0.11635771971526379, 'p': 0.11396490011492957, 'f': 0.11081786358373251}","{'r': 0.26494321886227257, 'p': 0.25644964866839565, 'f': 0.2522582965641118}"
6,1.193700,1.831726,"{'r': 0.27779184038770766, 'p': 0.27238584382856545, 'f': 0.26623911658164534}","{'r': 0.11543775596302285, 'p': 0.11458789859555832, 'f': 0.11070115164748713}","{'r': 0.26217339351793373, 'p': 0.2566857110375147, 'f': 0.25108550822262066}"
7,1.107400,1.855965,"{'r': 0.27943696292116865, 'p': 0.26986672477221824, 'f': 0.26569270088815505}","{'r': 0.11508534470655354, 'p': 0.1123345880961687, 'f': 0.10939412342240389}","{'r': 0.26382129505199836, 'p': 0.25437974054097207, 'f': 0.2506402723775001}"
8,1.032800,1.884968,"{'r': 0.27798351651973896, 'p': 0.2688245792094734, 'f': 0.264497834886212}","{'r': 0.11472621537088412, 'p': 0.11212697965469917, 'f': 0.10913152187957287}","{'r': 0.2626177556661765, 'p': 0.25355105467719924, 'f': 0.24966485547650238}"


***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64
Saving model checkpoint to checkpoint/freeze_pretrained_tf_test/checkpoint-17500
Configuration saved in checkpoint/freeze_pretrained_tf_test/checkpoint-17500/config.json
Model weights saved in checkpoint/freeze_pretrained_tf_test/checkpoint-17500/pytorch_model.bin
tokenizer config file saved in checkpoint/freeze_pretrained_tf_test/checkpoint-17500/tokenizer_config.json
Special tokens file saved in checkpoint/freeze_pretrained_tf_test/checkpoint-17500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64
Saving model checkpoint to checkpoint/freeze_pretrained_tf_test/checkpoint-35000
Configuration saved in checkpoint/freeze_pretrained_tf_test/checkpoint-35000/config.json
Model weights saved in checkpoint/freeze_pretrained_tf_test/checkpoint-35000/pytorch_model.bin
tokenizer config file saved in checkpoint/freeze_pretrained_tf_test/checkpoint-35000/tokenizer_config.json
Speci

TrainOutput(global_step=140000, training_loss=1.4254670200892856, metrics={'train_runtime': 26883.8391, 'train_samples_per_second': 104.149, 'train_steps_per_second': 6.509, 'total_flos': 1.7072133041553408e+17, 'train_loss': 1.4254670200892856, 'epoch': 8.0})

In [28]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 35004
  Batch size = 64


{'eval_loss': 1.784661054611206,
 'eval_rouge-1': {'r': 0.27932836236914305,
  'p': 0.2696070241183919,
  'f': 0.26546033966525173},
 'eval_rouge-2': {'r': 0.11674878219703241,
  'p': 0.11425068607060665,
  'f': 0.11103629017107475},
 'eval_rouge-l': {'r': 0.264616009780644,
  'p': 0.255111784242901,
  'f': 0.2513003640539408},
 'eval_runtime': 1199.1166,
 'eval_samples_per_second': 29.191,
 'eval_steps_per_second': 0.456,
 'epoch': 8.0}

In [29]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, num_beams=5,no_repeat_ngram_size=3,
                            attention_mask=attention_mask, 
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id,
                            eos_token_id=tokenizer.eos_token_id,)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

loading configuration file checkpoint/encoder_decoder(4)freeze_MLM_test/checkpoint-192500/config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Model config BartConfig {
  "_name_or_path": "checkpoint/encoder_decoder(4)freeze_MLM_test/checkpoint-192500",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 1,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3

In [30]:
#summaries_before_tuning = []
#for test_sample in tqdm(test_samples):
#    summaries_before_tuning.append(generate_summary(test_sample, model_before_tuning)[1])
#summaries_before_tuning = list(itertools.chain(*summaries_before_tuning))    
    
summaries_after_tuning=[]
for test_sample in tqdm(test_samples):
    summaries_after_tuning.append(generate_summary(test_sample, model)[1])
summaries_after_tuning = list(itertools.chain(*summaries_after_tuning))

  0%|                                                                                | 0/35004 [00:00<?, ?it/s]/home/jx7789/anaconda3/envs/dohwan/lib/python3.7/site-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 32 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
100%|██████████████████████████████████████████████████████████████████| 35004/35004 [1:20:18<00:00,  7.26it/s]


In [31]:
rouge.get_scores(summaries_after_tuning, test_samples["Summary"], avg=True)


{'rouge-1': {'r': 0.24189095941535405,
  'p': 0.26588413707005826,
  'f': 0.24452019570122735},
 'rouge-2': {'r': 0.09658707970609752,
  'p': 0.10892169666229526,
  'f': 0.09821027589125457},
 'rouge-l': {'r': 0.22957735149633765,
  'p': 0.25212164685894584,
  'f': 0.23195978246621796}}

In [32]:
for i in range(0, len(summaries_after_tuning), 1000):
    print('idx_{} '.format(i))
    #print("Summary before \n", summaries_before_tuning[i])
    print()
    print("Summary after \n", summaries_after_tuning[i])
    print()
    print("Target summary \n", test_samples["Summary"][i])
    print()
    print('Text', test_samples["Text"][i])
    print('-'*100)
    print()  

idx_0 

Summary after 
 영업팀 과장님이 보내줬는데 팀장님이 해줄지 모르겠고 저번에 부산 갈 때도 숙소로 엄청 싸웠다

Target summary 
 팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다

Text #일과 직업# 웅 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소로 엄청 싸워서 웅 흥 4개월가는거도아니고 3일가는데 좀해주지 거 얼마한다구 내말이 아니 해봤자 3일 다 합해도 몇만원 차이인데 너무해 그때는 2일이었는데도 만원 더 싼데에서 자꾸 자라고 넘 시러 일단
----------------------------------------------------------------------------------------------------

idx_1000 

Summary after 
 내일 아르바이트 면접이 4시 반이라서 혼자 밥 먹고 면접 보고 오겠다고 한다

Target summary 
 내일 채점 보조 아르바이트 면접이 4시 30분에 있다

Text #일과 직업# 내일 못 먹겠다 나 알바 면접 네시반이얌 그래서 거기 가야되는데 집에서 가면 넘 멀엉 혼자 밥먹고 알바 면접보고 오겠움여 어디 면접 알바면접 뭐 채점보조 알써 잘갔다와
----------------------------------------------------------------------------------------------------

idx_2000 

Summary after 
 인턴인데 아직 발표를 안 했다고 하자 발표를 왜 안 하냐고 한다

Target summary 
 인턴 발표가 나질 않는데 포스팅은 올라와 의문을 갖는다

Text #일과 직업# #@시스템#사진# 근데아직도 발표안함 다들똑같은입장 강 안했나봄 발표를 왜 안하지 인턴인데 그니까 근데 #@기타# 포스팅은 계속 올라놈 그뭐지 담당자가다를듯 포스팅올리눈사람따로 에휴 언제까지 기다려
------------------------

Target summary 
 가끔 학교 재학 증명서 발급해서 저장하는 프로그램같이 안 되는 것이 있지만 장점이 더 많아서 맥북을 사용한다고 한다

Text #개인 및 관계# 막 교수님이 뭐가 안 열리지 맥북에서는 안영린다고 공지했길래 신기해써 근데 나는 장점이 더 많아성 안되는거 가끔 학교 재학증명서 발급해서 저장하는 이땈 프로그램 아아
----------------------------------------------------------------------------------------------------

idx_27000 

Summary after 
 현지가이드가 잘생겼었는데 유부남이었고 사무장님은 왜 결혼 여부를 물어보는지 모르겠다

Target summary 
 현지 가이드가 잘생긴 유부남인데 사무장님이 결혼 여부를 물어서 불편했다

Text #개인 및 관계# 현지가이드가 잘생겼었어 아 존잼이겠는데 근데 유부남이었어 아쉘 쉣 사무장님은 왜 결혼 여부를 물어봐서 사람 불편하게 하냐 불편한 사람은 나임 앜개웃곀
----------------------------------------------------------------------------------------------------

idx_28000 

Summary after 
 점심을 늦게 먹어서 늦은잠을 잤다

Target summary 
 늦게 일어나서 점심을 늦게 먹었다고 하며 점심 겸 저녁이 아니라 늦은 점심이라고 말한다

Text #개인 및 관계# 왜캐 늦게멋엇냐 점심 늦게일어났우닠가 점저를 드셧네 모래 그게어케점저녀 점심이지 늦은잠심 세시에 먹음 점저아닙니까
----------------------------------------------------------------------------------------------------

idx_29000 

Summary after 
 온라인에서 팔면 사고 싶고 수정과처럼 따뜻하게 겨율에 먹고 싶다

Target summ